In [ ]:
import os
import numpy as np
import pandas as pd
import tifffile as tiff

def calculate_metrics(gt_mask, pred_mask):
    intersection = np.sum(gt_mask & pred_mask)
    union = np.sum(gt_mask | pred_mask)
    total_gt = np.sum(gt_mask)
    total_model = np.sum(pred_mask)
    true_positive = intersection
    false_positive = np.sum(pred_mask) - true_positive
    false_negative = np.sum(gt_mask) - true_positive
    true_negative = np.sum(~gt_mask & ~pred_mask)

    iou = intersection / union if union > 0 else np.nan
    dice = (2 * intersection) / (total_gt + total_model) if (total_gt + total_model) > 0 else np.nan
    accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else np.nan
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else np.nan
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else np.nan
    fpr = false_positive / (false_positive + true_negative) if (false_positive + true_negative) > 0 else np.nan
    fnr = false_negative / (false_negative + true_positive) if (false_negative + true_positive) > 0 else np.nan

    return iou, dice, accuracy, precision, recall, specificity, fpr, fnr

def process_image(img, gt_sem_path, gt_inst_path, pred_path, run_tag):
    gt_sem_files = sorted([f for f in os.listdir(os.path.join(gt_sem_path, img)) if f.endswith('.tiff')])
    # gt_inst_files = sorted([f for f in os.listdir(os.path.join(gt_inst_path, img)) if f.endswith('.tiff')])
    pred_files = sorted([f for f in os.listdir(os.path.join(pred_path, img)) if f.endswith('.tiff')])

    gt_sem_image = np.stack([tiff.imread(os.path.join(gt_sem_path, img, f)) for f in gt_sem_files])
    # gt_inst_image = np.stack([tiff.imread(os.path.join(gt_inst_path, img, f)) for f in gt_inst_files])
    pred_image = np.stack([tiff.imread(os.path.join(pred_path, img, f)) for f in pred_files])
    
    gt_sem_mask = gt_sem_image > 0
    pred_sem_mask = pred_image > 0
    
    iou, dice, accuracy, precision, recall, specificity, fpr, fnr = calculate_metrics(gt_sem_mask, pred_sem_mask)
    
    return [run_tag, img, iou, dice, accuracy, precision, recall, specificity, fpr, fnr]

def save_metrics(gt_sem_path, gt_inst_path, pred_path, results_path, run_tag, img_names):
    results = [process_image(img, gt_sem_path, gt_inst_path, pred_path, run_tag) for img in img_names]
    df = pd.DataFrame(results, columns=['run_tag', 'image_name', 'iou', 'dice', 'accuracy', 'precision', 'recall', 'specificity', 'fpr', 'fnr'])
    
    results_file = os.path.join(results_path, 'semantic_metrics.csv')
    
    if os.path.exists(results_file):
        existing_df = pd.read_csv(results_file)
        existing_df = existing_df.loc[:, ~existing_df.columns.str.contains('^Unnamed')]
        existing_df[['run_tag', 'image_name']] = existing_df[['run_tag', 'image_name']].astype(str)
        df[['run_tag', 'image_name']] = df[['run_tag', 'image_name']].astype(str)
        
        existing_df.set_index(['run_tag', 'image_name'], inplace=True)
        df.set_index(['run_tag', 'image_name'], inplace=True)
        
        existing_df.update(df)
        updated_df = pd.concat([existing_df, df[~df.index.isin(existing_df.index)]])
        updated_df.reset_index(inplace=True)
    else:
        updated_df = df
    
    updated_df.to_csv(results_file, index=False)
    return updated_df

In [3]:
import os

def setup_paths(dir_location, output_to_cloud, run_tag, is_original_data):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')
    
    base_gt_path = os.path.join(base_path, 'database')
    if is_original_data:
        gt_sem_path = os.path.join(base_gt_path, 'orignal_dataset', 'binary', 'tiff')
        gt_inst_path = os.path.join(base_gt_path, 'orignal_dataset', 'instance', 'tiff')
    else:
        gt_sem_path = os.path.join(base_gt_path, 'tablet_dataset', 'binary', 'tiff')
        gt_inst_path = os.path.join(base_gt_path, 'tablet_dataset', 'instance', 'tiff')

    base_output_path = os.path.join(base_path, 'outputs')
    pred_path = os.path.join(base_output_path, 'tiff', run_tag)
    if output_to_cloud:
        base_output_path = os.path.join(r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files', 'outputs')
    results_path = os.path.join(base_output_path, 'metrics', 'semantic_metrics')
    if not os.path.isdir(results_path):
        os.makedirs(results_path)

    print('Paths set')
    return gt_sem_path, gt_inst_path, pred_path, results_path

In [ ]:
img_names = ['2_Tablet']
# img_names = ['4_GenericD12']
# img_names = ['5_ClaritinD12']
# img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']
dir_location = 'refine'
# run_tag = 'pretrained_gen35_zscore4'
# run_tag = 'pretrained_clar35_zscore5'
run_tag = 'pretrained_tab40_gen35_clar35_fold0_e275244'
# run_tag = 'pretrained_tab40_gen35_clar35_foldsALL'

gt_sem_path, gt_inst_path, pred_path, results_path = setup_paths(dir_location, False, run_tag, False)
save_metrics(gt_sem_path, gt_inst_path, pred_path, results_path, run_tag, img_names)

Paths set


,run_tag,image_name,iou,dice,accuracy,precision,recall,specificity,fpr,fnr
0,pretrained_tab40_gen35_clar35_foldsALL,2_Tablet,0.550974,0.710487,0.956718,0.705305,0.715747,0.976032,0.023968,0.284253
1,pretrained_tab40_gen35_clar35_foldsALL,4_GenericD12,0.601633,0.751275,0.967669,0.988443,0.605895,0.999379,0.000621,0.394105
2,pretrained_tab40_gen35_clar35_foldsALL,5_ClaritinD12,0.663037,0.797381,0.975784,0.981203,0.671567,0.999017,0.000983,0.328433
3,pretrained_tab40_gen35_clar35_fold0,2_Tablet,0.606406,0.754984,0.965450,0.796727,0.717398,0.985330,0.014670,0.282602
4,pretrained_tab40_gen35_clar35_fold0,4_GenericD12,0.620958,0.766162,0.969103,0.982070,0.628078,0.998995,0.001005,0.371922
5,pretrained_tab40_gen35_clar35_fold0,5_ClaritinD12,0.680013,0.809533,0.976796,0.969289,0.694987,0.998318,0.001682,0.305013
6,pretrained_tab40_gen35_clar35_fold1,2_Tablet,0.535265,0.697294,0.957201,0.733678,0.664348,0.980672,0.019328,0.335652
7,pretrained_tab40_gen35_clar35_fold1,4_GenericD12,0.516825,0.681457,0.961010,0.997423,0.517516,0.999883,0.000117,0.482484
8,pretrained_tab40_gen35_clar35_fold1,5_ClaritinD12,0.580621,0.734675,0.970087,0.991046,0.583683,0.999597,0.000403,0.416317
9,pretrained_tab40_gen35_clar35_fold2,2_Tablet,0.562027,0.719613,0.957788,0.709491,0.730027,0.976043,0.023957,0.269973
